# Optimización de un modelo de Random Forest

Este notebook recoge los resultados de la búsqueda del mejor modelo de clasificación mediante Random Forest. El método entrena varios árboles de decisión (de clasificación en este caso) en submuestras de los datos y combina sus resultados para mejorar su precisión.

Para buscar el mejor modelo posible, se tratará de buscar los mejores hiperparámetros para:

* El número de árboles del bosque.
* La profundidad máxima que alcanzan estos.
* Función para evaluar una nueva división de una rama.

### Preparación de los datos

In [1]:
# Estructuras de datos
import pandas as pd
import numpy as np
# Data partition
from sklearn.model_selection import train_test_split
# Parameter tunning libraries
import optuna
from sklearn.model_selection import GridSearchCV
# Accuracy function
from sklearn.metrics import accuracy_score
# Model
from sklearn.ensemble import RandomForestClassifier

In [2]:
# Datos de entrenamiento
trainFNC = pd.read_csv("../data/train_FNC.csv")
trainSBM = pd.read_csv("../data/train_SBM.csv")
train_labels = pd.read_csv("../data/train_labels.csv")

# DataFrame con ambas fuentes de datos
train = pd.merge(left=trainFNC, right=trainSBM, left_on="Id", right_on="Id")
data = pd.merge(left=train_labels, right=train, left_on="Id", right_on="Id")
data.drop("Id", inplace=True, axis=1)

# Shuffle de los datos de train
data = data.sample(frac=1, random_state=0)
data.head(5)

,Class,FNC1,FNC2,FNC3,FNC4,FNC5,FNC6,FNC7,FNC8,FNC9,...,SBM_map55,SBM_map61,SBM_map64,SBM_map67,SBM_map69,SBM_map71,SBM_map72,SBM_map73,SBM_map74,SBM_map75
2,0,0.245850,0.216620,-0.124680,-0.353800,0.161500,-0.002032,-0.133020,-0.035222,0.259040,...,-0.257114,0.597229,1.220756,-0.059213,-0.435494,-0.092971,1.090910,-0.448562,-0.508497,0.350434
13,1,0.410730,-0.031925,0.210700,0.242260,0.320100,-0.419290,-0.187140,0.168450,0.599790,...,-0.050862,0.870602,0.609465,1.181878,-2.279469,-0.013484,-0.012693,-1.244346,-1.080442,-0.788502
53,1,0.070919,0.034179,-0.011755,0.019158,0.024645,-0.032022,0.004620,0.318170,0.212550,...,-1.539922,-1.495822,1.643866,1.687780,1.521086,-1.988432,-0.267471,0.510576,1.104566,-1.067206
41,0,0.087377,-0.052462,-0.007835,-0.112830,0.389380,0.216080,0.063572,-0.251230,-0.080568,...,-0.077353,-0.459463,-0.204328,-0.619508,-1.410523,-0.304622,-1.521928,0.593691,0.073638,-0.260920
74,0,0.202750,0.191420,-0.056662,-0.157780,0.244040,0.039780,-0.001503,0.001056,-0.048222,...,0.044457,0.593326,1.063052,0.434726,1.604964,-0.359736,0.210107,0.355922,0.730287,-0.323557


Vamos a usar la siguiente partición de los datos:

* 60% train $\sim$ 50 datos
* 20% validation $\sim$ 18 datos (se define al aplicar cross-validación en el ajuste)
* 20% test $\sim$ 18 datos

In [3]:
X = data.iloc[:, 1:]
y = data.iloc[:, 0]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print("Tamaño del dataset de train:", X_train.shape)
print("Tamaño del dataset de test:", X_test.shape)

Tamaño del dataset de train: (68, 410)
Tamaño del dataset de test: (18, 410)


In [ ]:
# Datos de test
testFNC = pd.read_csv("../data/test_FNC.csv")
testSBM = pd.read_csv("../data/test_SBM.csv")

# DataFrame con ambas fuentes de datos
test_kaggle = pd.merge(left=testFNC, right=testSBM, left_on="Id", right_on="Id")
test_kaggle.drop("Id", inplace=True, axis=1)
test_kaggle.head(5)

### Modelo

In [4]:
def train_model(model, param_grid):
    '''Función para realizar el entrenamiento y la búsqueda de hiperparámetros'''
    np.random.seed()
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=4)
    # cv = 4 porque así: el conjunto de validation tiene un 0.25 del tamaño de train y: 0.25 * 0.8 = 0.2 ~ 20% datos
    #                    el conjunto de train tiene un 0.75 del tamaño de train y: 0.75 * 0.8 = 0.6 ~ 60% datos
    grid_search.fit(X_train, y_train)
    
    print("Parámetros óptimos:", grid_search.best_params_)
    print("Modelo óptimo:", grid_search.best_estimator_)
    
    return grid_search.best_estimator_

Primera prueba de resultados, probando con valores concretos para el número de árboles y la profundidad de los mismos (parámetros ``n_estimators`` y ``max_depth`` respectivamente) y utilizando el método ``GridSearchCV`` de ``sklearn`` para realizar la búsqueda:

Segunda prueba, con un rango más amplio de valores:

In [6]:
# Definir y entrenar el modelo
model_RF = RandomForestClassifier(random_state=0)
param_grid_RF = {
    "n_estimators": range(50, 1050, 50),
    "criterion": ["gini", "entropy"],
    "max_depth": range(1, 21)
}
model_RF_opt = train_model(model_RF, param_grid_RF)

# Predicción en partición de test
y_pred_RF = model_RF_opt.predict(X_test)

# Precisión en partición de test
accuracy = accuracy_score(y_test, y_pred_RF)
print("Accuracy: {:0.2f}%".format(accuracy * 100))

Parámetros óptimos: {'criterion': 'entropy', 'max_depth': 4, 'n_estimators': 600}
Modelo óptimo: RandomForestClassifier(criterion='entropy', max_depth=4, n_estimators=600,
                       random_state=0)
Accuracy: 72.22%


Curiosamente, pudimos observar que si reducíamos el anterior espacio de búsqueda, podíamos obtener mejores resultados:

In [7]:
# Definir y entrenar el modelo
model_RF = RandomForestClassifier(random_state=0)
param_grid_RF2 = {
    "n_estimators": [100, 250, 500, 750, 1000],
    "criterion": ["gini", "entropy"],
    "max_depth": [5, 10, 15, 20, None]
}
model_RF_opt2 = train_model(model_RF, param_grid_RF2)

# Predicción en partición de test
y_pred_RF2 = model_RF_opt2.predict(X_test)

# Precisión en partición de test
accuracy = accuracy_score(y_test, y_pred_RF2)
print("Accuracy: {:0.2f}%".format(accuracy * 100))

Parámetros óptimos: {'criterion': 'entropy', 'max_depth': 5, 'n_estimators': 750}
Modelo óptimo: RandomForestClassifier(criterion='entropy', max_depth=5, n_estimators=750,
                       random_state=0)
Accuracy: 83.33%


La explicación a este suceso es que, pese a que hemos fijado una semilla en cada modelo en su declaración, una vez se ejecuta algo de código sobre ese modelo, el random state cambia. Esto unido a la inestabilidad del problema y los datos, que son muy sensibles a cualquier alteración dan lugar a este tipo de resultados.

En definitiva, fijar una semilla en estos casos lo que va a permitir es que los resultados de una celda, esto es un modelo concreto con unos hiperparámetros concretos, sean reproducibles.

La librería ``optuna`` es un framework específico para la optimización de hiperparámetros, repetiremos el proceso de búsqueda anterior utilizando esta librería en base a 2 métodos de búsqueda de hiperparámetros:

* **GridSampler:** equivalente a la anterior búsqueda de grid de sklearn. Lo usaremos para que los resultados sean comparables.
* **TPE:** algoritmo para hacer una "búsqueda inteligente" de hiperparámetros. Debería ahorrar intentos de combinaciones haciendo una selección inteligente de las pruebas. En nuestro caso le permitiremos probar un 10% del número de combinaciones posibles. 

In [8]:
def objectiveRF_Grid(trial):
    '''
    Define la función a optimizar por medio de un sampler de tipo GridSampler.
    En este caso se trata de maximizar el accuracy
    '''
    n_estimators =  trial.suggest_int("n_estimators", 50, 1000)
    criterion = trial.suggest_categorical("criterion", ["gini", "entropy"])
    max_depth = trial.suggest_int("max_depth", 1, 20)
    
    modelRF_optuna = RandomForestClassifier(criterion = criterion, max_depth = max_depth, n_estimators = n_estimators, 
                                            random_state=0)
    
    modelRF_optuna.fit(X_train, y_train)

    y_pred_RF_optuna = modelRF_optuna.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred_RF_optuna)
    return accuracy

In [9]:
# Prueba con GridSampler
optuna.logging.set_verbosity(optuna.logging.WARNING)

search_space = {
    "n_estimators": range(50, 1001, 50), 
    "criterion": ["gini", "entropy"], 
    "max_depth": range(1, 21)
}
sampler = optuna.samplers.GridSampler(search_space)
study_Grid = optuna.create_study(direction="maximize", sampler=sampler)
study_Grid.optimize(objectiveRF_Grid)

In [10]:
study_Grid.best_trial

FrozenTrial(number=7, values=[0.8888888888888888], datetime_start=datetime.datetime(2022, 6, 11, 23, 12, 10, 710916), datetime_complete=datetime.datetime(2022, 6, 11, 23, 12, 13, 897665), params={'n_estimators': 900, 'criterion': 'entropy', 'max_depth': 7}, distributions={'n_estimators': IntUniformDistribution(high=1000, low=50, step=1), 'criterion': CategoricalDistribution(choices=('gini', 'entropy')), 'max_depth': IntUniformDistribution(high=20, low=1, step=1)}, user_attrs={}, system_attrs={'search_space': OrderedDict([('criterion', ['entropy', 'gini']), ('max_depth', [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]), ('n_estimators', [50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000])]), 'grid_id': 137}, intermediate_values={}, trial_id=7, state=TrialState.COMPLETE, value=None)

In [11]:
# Definir y entrenar el modelo
modelRF_optuna_Grid = RandomForestClassifier(criterion="entropy", max_depth=7, n_estimators=900, random_state=0)  
modelRF_optuna_Grid.fit(X_train, y_train)

# Predicción en partición de test
y_pred_RF_optuna_Grid = modelRF_optuna_Grid.predict(X_test)

# Precisión en partición de test
accuracy = accuracy_score(y_test, y_pred_RF_optuna_Grid)
print("Accuracy: {:0.2f}%".format(accuracy * 100))

Accuracy: 88.89%


El método GridSampler no permite fijar semilla por lo que los resultados que devuelve serán diferentes en cuanto a parámetros aunque en cualquier caso tendrán la misma precisión en la partición de test.

Para probar TPE es necesario redefinir la función objetivo sobre la que se crea el estudio, ya que GridSampler definía dentro de la función el espacio en el que se tomaban los parámetros mientras que los valores exactos a probar los define fuera de la misma. Sin embargo, TPE define en la misma función objetivo los valores a tomar, por lo que hay que incluir los pasos.

In [28]:
def objectiveRF_TPE(trial):
    '''
    Define la función a optimizar por medio de un sampler de tipo TPE.
    En este caso se trata de maximizar el accuracy
    '''
    n_estimators =  trial.suggest_int("n_estimators", 50, 1000, 50) # optuna incluye en el rango el máximo y el mínimo
    criterion = trial.suggest_categorical("criterion", ["gini", "entropy"])
    max_depth = trial.suggest_int("max_depth", 1, 20)
    
    modelRF_optuna = RandomForestClassifier(criterion = criterion, max_depth = max_depth, n_estimators = n_estimators, 
                                            random_state=0)
    
    modelRF_optuna.fit(X_train, y_train)

    y_pred_RF_optuna = modelRF_optuna.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred_RF_optuna)
    return accuracy

In [13]:
# Prueba con TPE
optuna.logging.set_verbosity(optuna.logging.WARNING)

sampler = optuna.samplers.TPESampler(seed=0)  # Asegurar los reproducibilidad de los resultados
study_TPE = optuna.create_study(direction="maximize", sampler=sampler)
study_TPE.optimize(objectiveRF_TPE, n_trials=80)
# n_trials = (20 x 2 x 20) * 0.1 = 80

In [14]:
study_TPE.best_trial

FrozenTrial(number=2, values=[0.8888888888888888], datetime_start=datetime.datetime(2022, 6, 12, 0, 18, 30, 267412), datetime_complete=datetime.datetime(2022, 6, 12, 0, 18, 31, 790904), params={'n_estimators': 1000, 'criterion': 'entropy', 'max_depth': 11}, distributions={'n_estimators': IntUniformDistribution(high=1000, low=50, step=50), 'criterion': CategoricalDistribution(choices=('gini', 'entropy')), 'max_depth': IntUniformDistribution(high=20, low=1, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=2, state=TrialState.COMPLETE, value=None)

In [15]:
# Definir y entrenar el modelo
modelRF_optuna_TPE = RandomForestClassifier(criterion="entropy", max_depth=11, n_estimators=1000, random_state=0)  
modelRF_optuna_TPE.fit(X_train, y_train)

# Predicción en partición de test
y_pred_RF_optuna_TPE = modelRF_optuna_TPE.predict(X_test)

# Precisión en partición de test
accuracy = accuracy_score(y_test, y_pred_RF_optuna_TPE)
print("Accuracy: {:0.2f}%".format(accuracy * 100))

Accuracy: 88.89%


La librería ``optuna`` ha permitido obtener mejores resultados.

El código anterior, aunque realiza una búsqueda sobre el mismo rango de parámetros usados para el segundo intento con ``sklearn``, no está aplicando cross-validation an el entrenamiento. La siguiente celda sí lo implementa mediante el método ``OptunaSearchCV`` de ``optuna``:

In [16]:
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Definir y entrenar el modelo
model_RF = RandomForestClassifier(random_state=0)
param_grid_RF = {
    "n_estimators": optuna.distributions.IntUniformDistribution(50, 1000, 50),
    "criterion": optuna.distributions.CategoricalDistribution(["gini", "entropy"]),
    "max_depth": optuna.distributions.IntUniformDistribution(1, 20)
}

optuna_search = optuna.integration.OptunaSearchCV(model_RF, param_grid_RF, cv=4, n_trials=800, random_state=0)
# n_trials = 20 x 2 x 20 = 800
optuna_search.fit(X_train, y_train)

C:\Users\saral\AppData\Local\Temp\ipykernel_2928\2766480869.py:11: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  optuna_search = optuna.integration.OptunaSearchCV(model_RF, param_grid_RF, cv=4, n_trials=800, random_state=0)


OptunaSearchCV(cv=4, estimator=RandomForestClassifier(random_state=0),
               n_trials=800,
               param_distributions={'criterion': CategoricalDistribution(choices=('gini', 'entropy')),
                                    'max_depth': IntUniformDistribution(high=20, low=1, step=1),
                                    'n_estimators': IntUniformDistribution(high=1000, low=50, step=50)},
               random_state=0)

In [17]:
optunaCV_opt = optuna_search.best_estimator_
optunaCV_opt

RandomForestClassifier(criterion='entropy', max_depth=12, n_estimators=750,
                       random_state=0)

In [18]:
# Predicción en partición de test
y_pred_RF = optunaCV_opt.predict(X_test)

# Precisión en partición de test
accuracy = accuracy_score(y_test, y_pred_RF)
print("Accuracy: {:0.2f}%".format(accuracy * 100))

Accuracy: 83.33%


En este caso, aplicar cross-validation no mejora los resultados.

# Create submissions

In [ ]:
import pathlib
from datetime import datetime

def create_submission(pred, test_id=testFNC["Id"]):
    '''
    Función para generar un csv con las predicciones de un modelo para participar en la competición de Kaggle
    '''
    submissionDF = pd.DataFrame(list(zip(test_id, pred)), columns=["Id", "Probability"])
    print(submissionDF.shape) # Comprobación del tamaño, debe ser: (119748, 2)
    current_time = datetime.now().strftime("%d-%m-%Y_%Hh%Mmin")
    current_path = pathlib.Path().resolve()
    parent_path = current_path.parent
    submissionDF.to_csv(f"{parent_path}\submissions\MLSP_submission_RF_{current_time}.csv", header=True, index=False)

# Anexo

Para la optimización de los modelos con ``optuna``, se han probado dos métodos de "sampler": GridSampler y TPE, siendo este último algoritmo de especial interés ya que, si resulta ser efectivo, al hacer una búsqueda inteligente de los hiperparámetros más adecuados podría ahorrar mucho tiempo y recursos de computación respecto a cualquiera de las otras alternativas vistas en este trabajo.

Para comprobar la eficacia de este modelo vamos a realizar una prueba en la que compararemos 4 combinaciones de los hiperparámetros de manera aleatoria y 4 combinaciones (iteraciones) con este algoritmo de Optuna. Si TPE es eficaz, lo que se espera es que los resultados de este algoritmo mejoren los resultados de la búsqueda aleatoria y que cada iteración sea mejor que la anterior.

### Resultados prueba aleatoria

In [25]:
import random

random.seed(0)

random_params = {}
for key in param_grid_RF.keys():
    random_params[key] = random.choices(param_grid_RF[key], k=4)
    
random_params

{'n_estimators': [850, 800, 450, 300],
 'criterion': ['entropy', 'gini', 'entropy', 'gini'],
 'max_depth': [10, 12, 19, 11]}

In [26]:
for i in range(4):
    # Definir y entrenar el modelo
    modelRF_random = RandomForestClassifier(criterion=random_params["criterion"][i], max_depth=random_params["max_depth"][i], 
                                            n_estimators=random_params["n_estimators"][i], random_state=0)  
    modelRF_random.fit(X_train, y_train)

    # Predicción en partición de test
    y_pred_RF_random = modelRF_random.predict(X_test)

    # Precisión en partición de test
    accuracy = accuracy_score(y_test, y_pred_RF_random)
    print("Prueba aleatoria {} ------ Accuracy: {:0.2f}%".format(i, accuracy * 100))

Prueba aleatoria 0 ------ Accuracy: 88.89%
Prueba aleatoria 1 ------ Accuracy: 72.22%
Prueba aleatoria 2 ------ Accuracy: 83.33%
Prueba aleatoria 3 ------ Accuracy: 72.22%


### Resultados prueba con Optuna + TPE sampler

In [35]:
# Prueba con TPE
optuna.logging.set_verbosity(optuna.logging.INFO)

sampler = optuna.samplers.TPESampler(seed=0)  # Asegurar los reproducibilidad de los resultados
study_TPE = optuna.create_study(direction="maximize", sampler=sampler)
study_TPE.optimize(objectiveRF_TPE, n_trials=4)

[I 2022-06-19 15:32:09,324] A new study created in memory with name: no-name-672a364b-1857-4f1e-b409-a04a92b6924c
[I 2022-06-19 15:32:10,024] Trial 0 finished with value: 0.6666666666666666 and parameters: {'n_estimators': 550, 'criterion': 'gini', 'max_depth': 11}. Best is trial 0 with value: 0.6666666666666666.
[I 2022-06-19 15:32:10,607] Trial 1 finished with value: 0.6666666666666666 and parameters: {'n_estimators': 450, 'criterion': 'gini', 'max_depth': 18}. Best is trial 0 with value: 0.6666666666666666.
[I 2022-06-19 15:32:12,217] Trial 2 finished with value: 0.8888888888888888 and parameters: {'n_estimators': 1000, 'criterion': 'entropy', 'max_depth': 11}. Best is trial 2 with value: 0.8888888888888888.
[I 2022-06-19 15:32:13,180] Trial 3 finished with value: 0.6666666666666666 and parameters: {'n_estimators': 600, 'criterion': 'gini', 'max_depth': 2}. Best is trial 2 with value: 0.8888888888888888.


In [31]:
study_TPE.best_trial

FrozenTrial(number=2, values=[0.8888888888888888], datetime_start=datetime.datetime(2022, 6, 19, 15, 29, 42, 216973), datetime_complete=datetime.datetime(2022, 6, 19, 15, 29, 43, 671369), params={'n_estimators': 1000, 'criterion': 'entropy', 'max_depth': 11}, distributions={'n_estimators': IntUniformDistribution(high=1000, low=50, step=50), 'criterion': CategoricalDistribution(choices=('gini', 'entropy')), 'max_depth': IntUniformDistribution(high=20, low=1, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=2, state=TrialState.COMPLETE, value=None)